# Data Science Task 2

## Text Sentiment Analysis


### Importing required libraries

In [5]:
import nltk
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Reading csv

In [6]:
df = pd.read_csv('IMDB Dataset.csv', on_bad_lines='skip', engine="python")

### Download required NLP tools


In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

###  Text Preprocessing


In [8]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize words
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [9]:
# Apply preprocessing
df['cleaned_review'] = df['review'].apply(preprocess_text)

###  Convert sentimental lables to binary


In [11]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

### Split data into training and testing sets


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)

### Convert text data to numerical format using TF-IDF

In [13]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Model Training

In [14]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

### Predict Sentiment

In [15]:
y_pred = model.predict(X_test_tfidf)

### Evaluating Model

In [16]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

### Printing evaluation metrices

In [17]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.8701
Precision: 0.8487
Recall: 0.8879
F1-score: 0.8678


### Predict Sentiment

In [18]:
def predict_sentiment(text):
    processed_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([processed_text])
    prediction = model.predict(text_tfidf)
    return "Positive" if prediction[0] == 1 else "Negative"

In [19]:
test_review = "The movie was absolutely fantastic, with great acting!"
print("Predicted Sentiment:", predict_sentiment(test_review))

Predicted Sentiment: Positive
